In [ ]:
def train_validate(dataloader, model, criterion, optimizer, scheduler, 
                   dataloader_aug, model_aug, criterion_aug, optimizer_aug, scheduler_aug,
                   epoch, SUBEPOCH):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy  = 0.0
    best_model_wts_aug = copy.deepcopy(model_aug.state_dict())
    best_accuracy_aug  = 0.0
    
    for epoch in range(epoch_init, EPOCH):
        print('Epoch {}/{}'.format(epoch, EPOCH - 1))
        print('-' * 10)

    running_loss = 0.0
    running_correct = 0
    
    # Each epoch has a training and validation phase
    for phase in ['train', 'train_aug', 'val', 'val_aug']:
        if phase == 'train':
            #scheduler.step()
            model.train()  # Set model to training mode
        elif phase == 'train_aug':
            #scheduler_aug.step()
            model_aug.train()   # Set model to training mode
        elif phase == 'val':
            model.eval()  # Set model to evaluate mode
        else:
            model_aug.eval()   # Set model to evaluate mode


        # switch to train mode
        model.train()

        for i, (inputs, labels) in enumerate(train_loader):

            inputs = inputs.to(device) 
            labels = labels.to(device) 

            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # select the class with highest probability
            _, pred = outputs.max(1)
            running_correct += pred.eq(labels).sum().item()

            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            total_loss += loss.item()
            if i % SUBEPOCH == SUBEPOCH-1:    # print every SUBEPOCH mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / SUBEPOCH))
                #print(running_loss, SUBEPOCH, loss.item())
                running_loss = 0.0

        # Record statistics
        train_accuracy = 100.*running_correct/(len(train_loader)*BATCH_SIZE)
        train_loss = total_loss / len(train_loader)

        print('{:3d} epoch, training accuracy: {:.1f}%'.format(epoch+1, train_accuracy))
        print('           training loss    : {:.3f}'.format(train_loss))
        return train_accuracy, train_loss

    

In [ ]:
def train(train_loader, model, criterion, optimizer, scheduler, epoch, SUBEPOCH):
    #losses = AverageMeter()
    #top1 = AverageMeter()
    running_loss = 0.0
    total_loss = 0.0
    correct = 0
    
    # switch to train mode
    scheduler.step()
    model.train()
    
    for i, (inputs, labels) in enumerate(train_loader):
        
        inputs = inputs.to(device) 
        labels = labels.to(device) 

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # select the class with highest probability
        _, pred = outputs.max(1)
        correct += pred.eq(labels).sum().item()
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        total_loss += loss.item()
        if i % SUBEPOCH == SUBEPOCH-1:    # print every SUBEPOCH mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / SUBEPOCH))
            #print(running_loss, SUBEPOCH, loss.item())
            running_loss = 0.0

    # Record statistics
    train_accuracy = 100.*correct/(len(train_loader)*BATCH_SIZE)
    train_loss = total_loss / len(train_loader)
    
    print('{:3d} epoch, training accuracy: {:.1f}%'.format(epoch+1, train_accuracy))
    print('           training loss    : {:.3f}'.format(train_loss))
    return train_accuracy, train_loss

    

In [ ]:
def validate(val_loader, model, criterion):
    #losses = AverageMeter()
    #top1 = AverageMeter()
    #running_loss = 0.0
    total_loss = 0.0
    correct = 0
    
    # switch to train mode
    model.eval()
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):

            inputs = inputs.to(device) 
            labels = labels.to(device) 

            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # select the class with highest probability
            _, pred = outputs.max(1)
            correct += pred.eq(labels).sum().item()
            
            # print statistics
            total_loss += loss.item()
            
    # Record statistics
    val_accuracy = 100.*correct/(len(val_loader)*BATCH_SIZE)
    val_loss = total_loss / len(val_loader)
    
    print('           validation accuracy: {:.1f}%'.format(val_accuracy))
    print('           validation loss    : {:.3f}'.format(val_loss))
    return val_accuracy, val_loss
    

In [ ]:
def evaluate(eva_loader, model, criterion, classes):
    #losses = AverageMeter()
    #top1 = AverageMeter()
    #running_loss = 0.0
    total_loss = 0.0
    correct = 0
    
    correct_class = np.zeros(len(classes), dtype=np.int)
    error_class = np.zeros(len(classes), dtype=np.int)
    accu_class = np.zeros(len(classes))
    result = np.zeros((len(classes), len(classes)), dtype=np.int)
    
    
    # switch to train mode
    model.eval()
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(eva_loader):

            inputs = inputs.to(device) 
            labels = labels.to(device) 

            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # select the class with highest probability
            _, pred = outputs.max(1)
            correct += pred.eq(labels).sum().item()
            
            # print statistics
            total_loss += loss.item()
            prednp = pred.cpu().numpy()
            labelnp = labels.cpu().numpy()

            for idx in range(len(labelnp)):
                if prednp[idx] == labelnp[idx]:
                    correct_class[labelnp[idx]] += 1
                else:
                    error_class[labelnp[idx]] += 1
                    
                result[labelnp[idx]][prednp[idx]] += 1 
    
    #for idx in reversed(range(len(classes))):
    #    print('Class {:15s}: correct={}, error={}'.format(classes[idx], correct_class[idx], error_class[idx]))
    
    
    for idx in range(len(classes)):
        accu_class[idx] = correct_class[idx] / (correct_class[idx] + error_class[idx]) * 100
    
    # Record statistics
    eva_accuracy = 100.*correct/(len(eva_loader)*BATCH_SIZE)
    eva_loss = total_loss / len(eva_loader)
    
    print('           evaluation accuracy: {:.1f}%'.format(eva_accuracy))
    print('           evaluation loss    : {:.3f}'.format(eva_loss))
    print('Total evaluation accuracy is {:.2f}% and loss is {:.5f}'.format(eva_accuracy, eva_loss))
    print('For each class in Food-11:')
    for i in reversed(range(len(classes))):
        print('    Accuracy of {:15s} : {:.2f}%'.format(classes[i], accu_class[i]))
        
    show_class_accuracy(eva_accuracy, eva_loss, accu_class, classes)
    show_table_result(classes, np.flip(result))
    
    return eva_accuracy, eva_loss, accu_class
    